In [ ]:
pip install gym==0.21.0

In [ ]:
pip install pyglet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import PolynomialFeatures
import time
import gym

In [ ]:
gym.version.VERSION

In [ ]:
env = gym.make("CartPole-v0")

In [ ]:
s=env.reset()
s

In [ ]:
a=env.action_space.sample() #random action
a

In [ ]:
s_next, r, done, info = env.step(a)

In [ ]:
env.reset()
done=False
while not done:
    a=env.action_space.sample() #random action
    s_next, r, done, info = env.step(a)
    time.sleep(0.05)
    env.render()
env.close()

In [ ]:
def merge_state_action(s, a):
    return np.concatenate((s, [a]))

In [ ]:
s=env.reset()
a=env.action_space.sample()
sa=merge_state_action(s, a)
sa

In [ ]:
poly_features = PolynomialFeatures(degree=2)
poly_features.fit([sa])
poly_features.transform([sa])

In [ ]:
def gather_samples(n_episodes=10000):
    samples = []
    for i in range(n_episodes):
        s = env.reset()
        done=False
        while not done:
            a=env.action_space.sample() #random action
            sa = merge_state_action(s, a)
            samples.append(sa)
            s, r, done, info = env.step(a)
    return samples

In [ ]:
samples=gather_samples(n_episodes=10000)
samples

In [ ]:
from sklearn.kernel_approximation import Nystroem
nystrom_featurizer = Nystroem(gamma=1,random_state=1,n_components=100)
nystrom_featurizer.fit(samples)
nystrom_featurizer.components_

In [ ]:
rbf_feature = RBFSampler(n_components=100)
rbf_feature.fit(samples)

In [ ]:
def predictV(s,a,w,kernel):
    sa = merge_state_action(s, a)
    if kernel=='poly':
        poly_features.fit([sa])
        x=poly_features.transform([sa])[0]
    elif kernel=='rbf':
        x = rbf_feature.transform([sa])[0]
    else:
        x=nystrom_featurizer.transform([sa])[0]
    return np.dot(x,w)

In [ ]:
def gradientV(s,a,kernel):
    sa = merge_state_action(s, a)
    if kernel=='poly':
        poly_features.fit([sa])
        x=poly_features.transform([sa])[0]
    elif kernel=='rbf':
        x = rbf_feature.transform([sa])[0]
    else:
        x=nystrom_featurizer.transform([sa])[0]
    return x

In [ ]:
env.action_space.n

In [ ]:
def predictV_all_actions(s,w,kernel):
    values=[]
    for a in range(env.action_space.n):
        values.append(predictV(s,a,w,kernel))
    return values

In [ ]:
def epsilon_greedy(s, w, kernel, eps=0.1):
    values=[]
    p = np.random.random()
    if p < (1 - eps):
        values = predictV_all_actions(s,w,kernel)
        return np.argmax(values)
    else:
        return env.action_space.sample()

In [ ]:
GAMMA=0.9
ALPHA = 0.1
#Initialize weights as zero
#RBFSampler has 100 dimensions as default, hence we have 100 features and weights
w = np.zeros(poly_features.n_output_features_) 
n_episodes = 1500
for it in range(n_episodes):
    # begin a new episode
    s=env.reset()
    done=False
    while not done:
        a = epsilon_greedy(s, w, kernel='poly',eps=0.1)
        s2, r, done, info = env.step(a)
        if done:
            target = r
        else:
            values = predictV_all_actions(s2,w,kernel='poly')
            target = r + GAMMA * np.max(values)               
        # update the weights
        g = gradientV(s,a,kernel='poly')
        err=target - predictV(s,a,w,'poly')
        w += ALPHA * err * g
        # update state
        s = s2      

In [ ]:
w

In [ ]:
def watch_agent(w,kernel):
    done = False
    episode_reward = 0
    s = env.reset()
    while not done:
        a = epsilon_greedy(s, w, kernel,eps=0)
        s, r, done, info = env.step(a)
        time.sleep(0.05)
        env.render()
        episode_reward += r
    print("Episode reward:", episode_reward)

In [ ]:
watch_agent(w,kernel='poly')
env.close()

In [ ]:
GAMMA=0.9
ALPHA = 0.1
#Initialize weights as zero
w = np.zeros(100) 
n_episodes = 500
for it in range(n_episodes):
    print(it)
    # begin a new episode
    s=env.reset()
    done=False
    while not done:
        a = epsilon_greedy(s, w, kernel='nystrom',eps=0.1)
        s2, r, done, info = env.step(a)
        if done:
            target = r
        else:
            values = predictV_all_actions(s2,w,kernel='nystrom')
            target = r + GAMMA * np.max(values)               
        # update the weights
        g = gradientV(s,a,kernel='nystrom')
        err=target - predictV(s,a,w,'nystrom')
        w += ALPHA * err * g
        #print(w[0])
        # update state
        s = s2  

In [ ]:
watch_agent(w,kernel='nystrom')
env.close()

In [ ]:
GAMMA=0.9
ALPHA = 0.1
#Initialize weights as zero
w = np.zeros(100) 
n_episodes = 500
for it in range(n_episodes):
    print(it)
    # begin a new episode
    s=env.reset()
    done=False
    while not done:
        a = epsilon_greedy(s, w, kernel='rbf',eps=0.1)
        s2, r, done, info = env.step(a)
        if done:
            target = r
        else:
            values = predictV_all_actions(s2,w,kernel='rbf')
            target = r + GAMMA * np.max(values)               
        # update the weights
        g = gradientV(s,a,kernel='rbf')
        err=target - predictV(s,a,w,'rbf')
        w += ALPHA * err * g
        # update state
        s = s2  

In [ ]:
watch_agent(w,kernel='rbf')
env.close()